# Setup

## Imports

In [1]:
import pandas
import numpy
from copy import deepcopy
import pickle
from ipywidgets import ToggleButtons, Dropdown, SelectionSlider, FloatProgress, HTML, Label, interactive_output, HBox, VBox

## Load Models

In [2]:
path = '/content/drive/MyDrive/Colab Notebooks/QMIND_2023-2024/'
model_multiclass = pickle.load(open(path + 'model_multiclass.sav', 'rb'))
model_binary = pickle.load(open(path + 'model_binary.sav', 'rb'))
model_regression = pickle.load(open(path + 'model_regression.sav', 'rb'))

multiclass_mapping = {1: '1-6 days', 2: '7-14 days', 3: '15-119 days'}
binary_mapping = {0: '1-10 days', 1: '11-119 days'}

## App Setup

In [3]:
description_simple_mapping = {'Nervous System': 'diseases and disorders of the nervous system',
                              'Circulatory System': 'diseases and disorders of the circulatory system',
                              'Digestive System': 'diseases and disorders of the digestive system',
                              'Musculoskeletal System & Connective Tissue': 'diseases and disorders of the musculoskeletal system and connective tissue',
                              'Respiratory System': 'diseases and disorders of the respiratory system',
                              'Infection & Parasite': 'infectious and parasitic diseases (systemic or unspecified sites)',
                              'Mental Disease': 'mental diseases and disorders',
                              'Kidney & Urinary Tract': 'diseases and disorders of the kidney and urinary tract',
                              'Hepatobiliary System & Pancreas': 'diseases and disorders of the hepatobiliary system and pancreas',
                              'Rehabilitation, Aftercare, & Other': 'rehabilitation, aftercare, other factors influencing health status and other health service contacts',
                              'Endocrine, Nutritional, & Metabolic': 'endocrine, nutritional and metabolic diseases and disorders',
                              'Skin, Subcutaneous Tissue, & Breast': 'diseases and disorders of the skin, subcutaneous tissue and breast',
                              'Lymphatic, Hematopoietic, Other Malignancy, Chemotherapy, & Radiotherapy': 'lymphatic, hematopoietic, other malignancies, chemotherapy and radiotherapy',
                              'Blood/Blood Forming Organs & Immunological': 'diseases and disorders of the blood and blood forming organs and immunological disorders',
                              'Poisoning/Toxic Effects, Other Injuries, & Other Treatment Complications': 'poisonings, toxic effects, other injuries and other complications of treatment',
                              'Ear, Nose, Mouth, Throat, & Craniofacial': 'ear, nose, mouth, throat and craniofacial diseases and disorders',
                              'Alcohol/Drug Use & Alcohol/Drug Induced Organic Mental Disease': 'alcohol/drug use and alcohol/drug induced organic mental disorders',
                              'Newborn & Other Neonate With Perinatal Condition': 'newborns and other neonates with conditions originating in the perinatal period',
                              'Myeloproliferative & Poorly Differentiated Neoplasm': 'myeloproliferative diseases and disorders, and poorly differentiated neoplasm',
                              'Pregnancy, Childbirth, & Puerperium': 'pregnancy, childbirth and the puerperium',
                              'Multiple Significant Trauma': 'multiple significant trauma',
                              'Female Reproductive System': 'diseases and disorders of the female reproductive system',
                              'Human Immunodeficiency Virus': 'human immunodeficiency virus infections',
                              'Male Reproductive System': 'diseases and disorders of the male reproductive system',
                              'Eye': 'diseases and disorders of the eye',
                              'Burn': 'burns'
                             }

dictionary = {'Age Group': ['0 to 17', '18 to 29', '30 to 49', '50 to 69', '70 or Older'],
              'Gender': ['M', 'F'],
              'Race': ['White', 'Other Race', 'Black/African American', 'Multi-racial'],
              'Ethnicity': ['Not Span/Hispanic', 'Spanish/Hispanic', 'Unknown', 'Multi-ethnic'],
              'Type of Admission': ['Emergency', 'Elective', 'Urgent', 'Trauma', 'Newborn'],
              'APR MDC Description': list(description_simple_mapping.values()),
              'APR Severity of Illness Description': ['Minor', 'Moderate', 'Major', 'Extreme'],
              'APR Risk of Mortality': ['Minor', 'Moderate', 'Major', 'Extreme'],
              'APR Medical Surgical Description': ['Medical', 'Surgical']
             }

In [6]:
# Inputs
age_group = ToggleButtons(options=dictionary['Age Group'], description='Age:', style={'button_width': 'auto'})
gender = ToggleButtons(options=dictionary['Gender'], description='Gender:', style={'button_width': 'auto'})
race = ToggleButtons(options=dictionary['Race'], description='Race:', style={'button_width': 'auto'})
ethnicity = ToggleButtons(options=dictionary['Ethnicity'], description='Ethnicity:', style={'button_width': 'auto'})
type_of_admission = ToggleButtons(options=dictionary['Type of Admission'], description='Admission:', style={'button_width': 'auto'})
description = Dropdown(options=description_simple_mapping.keys(), description='Description:', layout={'width': 'initial'})
severity_of_illness = SelectionSlider(options=dictionary['APR Severity of Illness Description'], description='Severity:', continuous_update=True)
risk_of_mortality = SelectionSlider(options=dictionary['APR Risk of Mortality'], description='Mortality Risk:', continuous_update=True, style={'description_width': 'initial'})
medical_surgical = ToggleButtons(options=dictionary['APR Medical Surgical Description'], description='Type:', style={'button_width': 'auto'})

# Outputs
binary_label = HTML(value='<b>Binary:</b>')
binary_1_label = Label(value='')
binary_2_label = Label(value='')
binary_1 = FloatProgress(value=0, min=0, max=1, style={'description_width': 'initial'})
binary_2 = FloatProgress(value=0, min=0, max=1, style={'description_width': 'initial'})

regression_label = HTML(value='<b>Regression:</b>')
regression = Label(value='')

multiclass_label = HTML(value='<b>Multi-class:</b>')
multiclass_1_label = Label(value='')
multiclass_2_label = Label(value='')
multiclass_3_label = Label(value='')
multiclass_1 = FloatProgress(value=0, min=0, max=1, style={'description_width': 'initial'})
multiclass_2 = FloatProgress(value=0, min=0, max=1, style={'description_width': 'initial'})
multiclass_3 = FloatProgress(value=0, min=0, max=1, style={'description_width': 'initial'})

# Interactivity function
def run(age_group, gender, race, ethnicity, type_of_admission, description, severity_of_illness, risk_of_mortality, medical_surgical):
    user_input = {'Age Group': age_group, 'Gender': gender, 'Race': race, 'Ethnicity': ethnicity, 'Type of Admission': type_of_admission,
                  'APR MDC Description': description_simple_mapping[description], 'APR Severity of Illness Description': severity_of_illness,
                  'APR Risk of Mortality': risk_of_mortality, 'APR Medical Surgical Description': medical_surgical
                 }

    new = {}
    for key in dictionary.keys():
        for value in numpy.sort(dictionary[key]):
            if user_input[key] == value: new[key + ' - ' + value] = 1
            else: new[key + ' - ' + value] = 0
    new = pandas.DataFrame(data=new, index=[0])

    multiclass_probabilities = model_multiclass.predict_proba(new)[0]
    binary_probabilities = model_binary.predict_proba(new)[0]
    regression_output = numpy.exp(model_regression.predict(new)[0])

    binary_1_label.value = binary_mapping[0] + ' (' + str(round(100*binary_probabilities[0])) + '%)'
    binary_2_label.value = binary_mapping[1] + ' (' + str(round(100*binary_probabilities[1])) + '%)'
    binary_1.value = binary_probabilities[0]
    binary_1.bar_style = 'success'
    binary_2.value = binary_probabilities[1]
    binary_2.bar_style = 'danger'

    regression.value = str(round(regression_output)) + ' day(s)'

    multiclass_1_label.value = multiclass_mapping[1] + '\t(' + str(round(100*multiclass_probabilities[0])) + '%)'
    multiclass_2_label.value = multiclass_mapping[2] + '\t(' + str(round(100*multiclass_probabilities[1])) + '%)'
    multiclass_3_label.value = multiclass_mapping[3] + '\t(' + str(round(100*multiclass_probabilities[2])) + '%)'
    multiclass_1.value = multiclass_probabilities[0]
    multiclass_1.bar_style = 'success'
    multiclass_2.value = multiclass_probabilities[1]
    multiclass_2.bar_style = 'warning'
    multiclass_3.value = multiclass_probabilities[2]
    multiclass_3.bar_style = 'danger'

# Interaction and app
out = interactive_output(run, {'age_group': age_group, 'gender': gender, 'race': race, 'ethnicity': ethnicity, 'type_of_admission': type_of_admission,
                               'description': description, 'severity_of_illness': severity_of_illness, 'risk_of_mortality': risk_of_mortality,
                               'medical_surgical': medical_surgical})
inputs = VBox([age_group, gender, race, ethnicity, type_of_admission, medical_surgical, description, severity_of_illness, risk_of_mortality])
outputs = VBox([binary_label, HBox([VBox([binary_1_label, binary_2_label]), VBox([binary_1, binary_2])]), regression_label, regression, multiclass_label, HBox([VBox([multiclass_1_label, multiclass_2_label, multiclass_3_label]), VBox([multiclass_1, multiclass_2, multiclass_3])])])
app = HBox([inputs, outputs])

# App

In [7]:
display(app)